## Pandas (High Perfomance Pandas)

Una de las potencias de Numpy y Pandas es, por una lado las operaciones vectorizadas de Numpy y las operaciones de agregación de Pandas, que son bastante eficaces y efectivas. 

Estas, están basadas, a menudo, en la creación de objetos temporales intermedios, que pueden llegar a causar sobrecostes en términos de cálculo y uso de memoria.

Pandas, proporciona una serie de funciones que te permiten acceder a operaciones directas en C, sin el coste de generación de objetos intermedios. Estas funciones son eval() y query(), que se encuentran en el Numexpr paquete.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

### Motivating query() and eval(): Compound Expressions

Numpy y Pandas soportan rápidas operaciones vectorizadas

In [2]:
rng = np.random.RandomState(42)

In [3]:
x = rng.rand(1000000)
y = rng.rand(1000000)

In [4]:
x.shape

(1000000,)

Está mas optimizada la operación vectorizada que realizar un bucle

In [5]:
%timeit x + y

1.43 ms ± 69.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x,y)), dtype=x.dtype, count=len(x))

142 ms ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Cuando estamos realizando operaciones sobre los arrays, llega a ser menos eficiente

In [7]:
mask = (x > 0.5) & (y < 0.5)

Seria algo así
    - tmp1 = (x > 0.5)
    - tmp2 = (y < 0.5)
    - mask = tmp1 & tmp2
    
Crearía temporalmente un array temporal por cada uno de los pasos, que en el caso que este sea muy grande, supondría un coste grande en recursos.

La librería Numexpr nos da la posibilidad de realizar esta operación elemento por elemento, sin la necesidad de generar arrays completos intermedios.    

#### Pandas - Eval()

Utiliza expresiones en strings para calcular eficientemente operaciones usando DataFrames

In [3]:
nrows = 100000

In [4]:
ncolumns = 100

In [5]:
rng = np.random.RandomState(42)

In [20]:
# Creamos 4 dataframes
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncolumns)) for i in range(4))

In [21]:
# Sumamos los 4
%time test1 = df1 + df2 + df3 + df4

CPU times: user 112 ms, sys: 136 ms, total: 248 ms
Wall time: 51.3 ms


In [22]:
# Sumamos los 4 utilizando la función eval()
%time  test2 = pd.eval('df1 + df2 + df3 + df4')

CPU times: user 96 ms, sys: 52 ms, total: 148 ms
Wall time: 39.8 ms


La función eval() es sobre el 50% más rápida

In [23]:
np.allclose(test1, test2)

True

In [24]:
# Creamos 5 dataframes
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100,3))) for i in range(5))

In [26]:
result1 = -df1 * df2 / (df3+ df4) - df5

In [27]:
result2 = pd.eval('-df1 * df2 / (df3+ df4) - df5')

In [28]:
np.allclose(result1, result2)

True

In [30]:
result1 = (df1 < df2) &  (df3 <= df4)

In [32]:
result2 = pd.eval('(df1 < df2) &  (df3 <= df4)')

In [33]:
np.allclose(result1, result2)

True

In [35]:
df = pd.DataFrame(rng.rand(1000,3), columns=['A', 'B', 'C'])

In [50]:
result1 = (df['A'] + df['B'] / df['C'] - 1)

In [43]:
result2 = pd.eval("(df.A + df.B / df.C - 1)")

In [44]:
np.allclose(result1, result2)

True

In [51]:
result3 = df.eval('A + B / C - 1')

In [52]:
np.allclose(result1, result3)

True

#### query()

In [53]:
result1 = df[(df['A'] < 0.5) & (df['B'] < 0.5)]

In [55]:
result2 = df.query('A < 0.5 and B < 0.5')

In [56]:
np.allclose(result1, result2)

True

### Cuando utilizar estas funciones

Hay dos consideraciones: 
* Tiempo de computación
* Uso de memoria

El uso de memoria es el aspecto más predecible de los dos, ya que se puede calcular el tamaño en memoria del array, y en función de los pasos temporales que se ejecutan, calcular aproximadamente el consumo de memoria. 

Si el tamaño es excesivamente grande, es una buena idea el uso de eval() o query()

En términos de tiempo de computación, no es demasiada significante la diferencia entre el uso de los métodos tradicionales y estos nuevos. 

Por lo tanto, el benificio del uso de estas funciones es para el ahorro de memoria, y algunas veces, que ofrecen una sintáxis del código más limpia.


